In [7]:
import os
from tqdm.notebook import tqdm
import polars as pl
import json


In [8]:
# Existing dataset dir
data_dir = './data/'

# Existing plots dir
plots_dir = data_dir+'/plots/'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)



In [9]:
def process_swaps(file_name, perdiod = "all"):
    # Load the parquet file
    swaps_df = pl.scan_parquet(file_name).collect(streaming=True)

    if (perdiod == "after") :
        start_date = pl.datetime(2024,3,13)
        swaps_df = swaps_df.filter((pl.col('block_timestamp') > start_date))

    if (perdiod == "before") :
        start_date = pl.datetime(2024,3,13)
        swaps_df = swaps_df.filter((pl.col('block_timestamp') < start_date))

    # Calculate required metrics
    # Calculate required metrics
    total_volume_USD = abs(swaps_df['amount_USD']).sum()
    swaps = swaps_df['transaction_hash'].shape[0]
    avg_volume_USD = total_volume_USD/swaps
    block_numbers = swaps_df['block_number'].n_unique()
    avgInBlock = swaps / block_numbers

    std_volume_USD = abs(swaps_df['amount_USD']).std()
    median_volume_USD = abs(swaps_df['amount_USD']).median()

    # Create formatted string
    my_row = f"\\num{{{swaps}}} & \\num{{{total_volume_USD/1000000:.2f}}} & \\num{{{avg_volume_USD:.2f}}} & \\num{{{std_volume_USD:.2f}}}& \\num{{{median_volume_USD:.2f}}}"
    return my_row

def process_swaps_fee(file_name, perdiod = "all", lp_fee=5):
    # Load the parquet file
    swaps_df = pl.scan_parquet(file_name).collect(streaming=True)

    if (perdiod == "after") :
        start_date = pl.datetime(2024,3,13)
        swaps_df = swaps_df.filter((pl.col('block_timestamp') > start_date))

    if (perdiod == "before") :
        start_date = pl.datetime(2024,3,13)
        swaps_df = swaps_df.filter((pl.col('block_timestamp') < start_date))

    # Calculate required metrics

    swaps = swaps_df['transaction_hash'].shape[0]
    sum_volume_USD = abs(swaps_df['amount_USD']).sum()

    sum_total_fee_USD = abs(swaps_df['total_fee_USD']).sum()
    sum_gas_fee_USD = abs(swaps_df['gas_fee_USD']).sum()
    sum_LP_fee_USD = abs(swaps_df['LP_fee_USD']).sum()
    sum_slippage_USD = abs(swaps_df['total_slippage_USD']).sum()

    avg_total_fee_USD = sum_total_fee_USD / swaps
    bps_total_fee_USD = sum_total_fee_USD / sum_volume_USD * 10000
    avg_gas_fee_USD = sum_gas_fee_USD / swaps
    bps_gas_fee_USD = sum_gas_fee_USD / sum_volume_USD * 10000
    avg_LP_fee_USD = sum_LP_fee_USD / swaps
    bps_LP_fee_USD = sum_LP_fee_USD / sum_volume_USD * 10000
    avg_slippage_USD = sum_slippage_USD / swaps
    bps_slippage_USD = sum_slippage_USD / sum_volume_USD * 10000

    median_volume_USD = abs(swaps_df['amount_USD']).mean()
    avg_block_impact = abs(swaps_df['block_impact']).mean() * median_volume_USD
    bps_block_impact = abs(swaps_df['block_impact']).mean() * 10000
    avg_price_impact = abs(swaps_df['price_impact']).mean() * median_volume_USD
    bps_price_impact = abs(swaps_df['price_impact']).mean()  * 10000

    # Create formatted string
    my_row = f" & \\num{{{avg_total_fee_USD:.2f}}} ({bps_total_fee_USD:.0f}bps) & \\num{{{avg_gas_fee_USD:.2f}}} ({bps_gas_fee_USD:.0f}bps) & \\num{{{avg_LP_fee_USD:.2f}}} ({bps_LP_fee_USD:.0f}bps) & \\num{{{avg_slippage_USD:.2f}}} ({bps_slippage_USD:.0f}bps) & - \\num{{{avg_block_impact:.2f}}} ({bps_block_impact:.1f}bps) & \\num{{{avg_price_impact:.2f}}} ({bps_price_impact:.1f}bps) "
    return my_row


In [10]:
file_name = "./data/swaps-ethereum-Uniswap-v3-WETH-USDC.paraquet"
result = process_swaps_fee(file_name)
print(result)
result = process_swaps_fee(file_name, "before")
print(result)
result = process_swaps_fee(file_name, "after")
print(result)

 & \num{97.20} (21bps) & \num{37.13} (8bps) & \num{23.11} (5bps) & \num{65.33} (14bps) & - \num{0.00} (0.0bps) & \num{4.33} (0.9bps) 
 & \num{92.96} (22bps) & \num{39.30} (9bps) & \num{20.94} (5bps) & \num{59.43} (14bps) & - \num{0.00} (0.0bps) & \num{3.55} (0.8bps) 
 & \num{102.26} (20bps) & \num{34.56} (7bps) & \num{25.69} (5bps) & \num{72.35} (14bps) & - \num{0.00} (0.0bps) & \num{5.37} (1.0bps) 


In [11]:
file_name = "./data/swaps-arbitrum-Uniswap-v3-WETH-USDC.paraquet"
result = process_swaps_fee(file_name)
print(result)
result = process_swaps_fee(file_name, "before")
print(result)
result = process_swaps_fee(file_name, "after")
print(result)

 & \num{2.46} (8bps) & \num{0.65} (2bps) & \num{1.51} (5bps) & \num{1.82} (6bps) & - \num{0.00} (0.0bps) & \num{0.05} (0.2bps) 
 & \num{9.68} (10bps) & \num{4.16} (4bps) & \num{4.73} (5bps) & \num{5.76} (6bps) & - \num{0.00} (0.0bps) & \num{0.36} (0.4bps) 
 & \num{2.00} (8bps) & \num{0.42} (2bps) & \num{1.31} (5bps) & \num{1.57} (6bps) & - \num{0.00} (0.0bps) & \num{0.04} (0.2bps) 


In [12]:
file_name = "./data/swaps-arbitrum-Uniswap-v3-WETH-USDCe.paraquet"
result = process_swaps_fee(file_name)
print(result)
result = process_swaps_fee(file_name, "before")
print(result)
result = process_swaps_fee(file_name, "after")
print(result)

 & \num{5.55} (15bps) & \num{3.41} (9bps) & \num{1.81} (5bps) & \num{2.18} (6bps) & - \num{0.00} (0.0bps) & \num{0.11} (0.3bps) 
 & \num{6.28} (11bps) & \num{3.08} (6bps) & \num{2.77} (5bps) & \num{3.30} (6bps) & - \num{0.00} (0.0bps) & \num{0.20} (0.4bps) 
 & \num{5.05} (22bps) & \num{3.64} (16bps) & \num{1.15} (5bps) & \num{1.41} (6bps) & - \num{0.00} (0.0bps) & \num{0.06} (0.3bps) 


In [13]:
file_name = "./data/swaps-base-Uniswap-v3-WETH-USDC.paraquet"
result = process_swaps_fee(file_name)
print(result)
result = process_swaps_fee(file_name, "before")
print(result)
result = process_swaps_fee(file_name, "after")
print(result)

 & \num{1.76} (10bps) & \num{0.52} (3bps) & \num{0.89} (5bps) & \num{1.24} (7bps) & - \num{0.00} (0.0bps) & \num{0.09} (0.5bps) 
 & \num{0.43} (22bps) & \num{0.11} (6bps) & \num{0.10} (5bps) & \num{0.36} (18bps) & - \num{0.00} (0.0bps) & \num{0.03} (1.4bps) 
 & \num{1.99} (10bps) & \num{0.59} (3bps) & \num{1.02} (5bps) & \num{1.40} (7bps) & - \num{0.00} (0.0bps) & \num{0.07} (0.4bps) 


In [14]:
file_name = "./data/swaps-optimism-Uniswap-v3-WETH-USDC.paraquet"
result = process_swaps_fee(file_name)
print(result)
result = process_swaps_fee(file_name, "before")
print(result)
result = process_swaps_fee(file_name, "after")
print(result)

 & \num{1.33} (11bps) & \num{0.31} (2bps) & \num{0.61} (5bps) & \num{1.10} (9bps) & - \num{0.00} (0.0bps) & \num{0.09} (0.7bps) 
 & \num{1.55} (10bps) & \num{0.36} (2bps) & \num{0.78} (5bps) & \num{1.27} (8bps) & - \num{0.00} (0.0bps) & \num{0.10} (0.7bps) 
 & \num{1.17} (12bps) & \num{0.26} (3bps) & \num{0.49} (5bps) & \num{0.97} (10bps) & - \num{0.00} (0.0bps) & \num{0.08} (0.8bps) 


In [15]:
file_name = "./data/swaps-zkSync-Uniswap-v3-WETH-USDC.paraquet"
result = process_swaps_fee(file_name, lp_fee=30)
print(result)
result = process_swaps_fee(file_name, "before", 30)
print(result)
result = process_swaps_fee(file_name, "after", 30)
print(result)

 & \num{0.87} (50bps) & \num{0.25} (15bps) & \num{0.52} (30bps) & \num{0.65} (38bps) & - \num{0.00} (0.0bps) & \num{0.04} (2.4bps) 
 & \num{1.37} (49bps) & \num{0.37} (13bps) & \num{0.83} (30bps) & \num{1.03} (37bps) & - \num{0.00} (0.0bps) & \num{0.06} (2.1bps) 
 & \num{0.32} (56bps) & \num{0.12} (21bps) & \num{0.17} (30bps) & \num{0.24} (40bps) & - \num{0.00} (0.0bps) & \num{0.02} (2.7bps) 


In [16]:
file_name = "./data/swaps-base-Uniswap-v3-WETH-USDC.paraquet"
swaps_df = pl.scan_parquet(file_name).collect(streaming=True)

# Create formatted string
#my_row = f"\\num{{{total_volume_USD}}} & \\num{{{swaps}}} & \\num{{{avg_volume_USD}}} & \\num{{{std_volume_USD}}}& \\num{{{median_volume_USD:.2f}}}"
#my_row

In [17]:

swaps_df

transaction_hash,transaction_index,log_index,block_number,block_timestamp,gas_price,amount_ETH,amount_USD,sqrtPriceX96,tick,liquidity,effective_price,spot_price,reserve_ETH,reserve_USD,CEX_price,gas_fee_gwei,gas_fee_USD,LP_fee_USD,total_slippage_USD,total_fee_USD,price_impact,volume_USD,block_slippage_USD,block_impact,total_fees
str,i64,i64,i64,datetime[μs],i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0x9a3d769c9149…",7,40,8638929,2024-01-01 00:00:05,1100055,-0.000942,2.150611,3.7843e24,-198995,2.9579e16,2282.53542,2281.396886,619.272645,1.4128e6,2281.65,3374.56172,0.0077,0.001075,null,null,0.000002,2.150611,0.0,0.0,0.5294
"""0x59b2d13ba831…",5,17,8638941,2024-01-01 00:00:29,1200061,0.013515,-30.817117,3.7842e24,-198996,2.9579e16,2280.206393,2281.29736,619.286153,1.4128e6,2281.19,718.243709,0.001638,0.015409,-0.01609,0.000957,0.000022,30.817117,0.0,0.0,0.035401
"""0x8e1edc43003e…",3,5,8638990,2024-01-01 00:02:07,5695,0.001,-2.280153,3.7842e24,-198996,2.9579e16,2280.153,2281.289997,619.287153,1.4128e6,2283.47,0.918922,0.000002,0.00114,-0.001144,-0.000002,0.000002,2.280153,0.0,0.0,0.498649
"""0x005b59dc28c2…",4,53,8639044,2024-01-01 00:03:55,10000,-0.026278,60.0,3.7850e24,-198991,2.9579e16,2283.303265,2282.258471,619.155742,1.4131e6,2285.69,185.52674,0.000424,0.03,0.042725,0.073149,0.000042,60.0,0.0,0.0,0.017413
"""0x005b59dc28c2…",4,26,8639044,2024-01-01 00:03:55,10000,-0.026284,60.0,3.7845e24,-198994,2.9579e16,2282.721877,2281.677362,619.234582,1.4129e6,2285.69,185.52674,0.000424,0.03,0.032545,0.062969,0.000042,60.0,60.0,1.8034e-9,0.017409
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0x50313da7cd6c…",1,5,13866120,2024-04-30 23:59:47,739651314,-0.191372,576.748554,4.3484e24,-196216,2.5013e18,3013.749881,3012.25565,45573.510067,1.3728e8,3011.6,110329.348601,0.332268,0.288374,0.281105,0.901747,0.000004,576.748554,0.0,0.0,0.002591
"""0x50fb9d88d8d7…",4,10,13866120,2024-04-30 23:59:47,49481257,-0.1917,577.740655,4.3484e24,-196216,2.5013e18,3013.775211,3012.280991,45573.318367,1.3728e8,3011.6,46233.158847,0.139236,0.28887,0.29615,0.724256,0.000004,577.740655,576.748554,1.7681e-11,0.002586
"""0x7b052d9ed1da…",17,75,13866120,2024-04-30 23:59:47,32957389,-0.02074,62.506288,4.3484e24,-196216,2.5013e18,3013.789298,3012.283733,45573.297627,1.3728e8,3011.6,12248.053346,0.036886,0.031253,0.031808,0.099947,4.5532e-7,62.506288,1154.489209,3.8292e-12,0.024087


In [ ]:
swaps_df['amount_USD']